In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import time
import copy
import os

In [2]:
import logging
l = logging.getLogger(__name__)

logging.basicConfig(format='[%(asctime)s][%(levelname)s]: %(message)s',
                        level=logging.INFO)

In [3]:
# Preprocessing according to the pre-trained model
# http://pytorch.org/docs/master/torchvision/models.html
preprocess = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
       mean=[0.485, 0.456, 0.406],
       std=[0.229, 0.224, 0.225]
    ),
])
    
def prepare_image(image):
    """
    Prepare image to be fed into the model.
    
    
    """
    
    # These preprocessing steps are taken from PyTorch's ImageNet example
    return preprocess(image).unsqueeze_(0)

In [4]:
ds_train = datasets.ImageFolder("./transfer_learning/training", preprocess)

In [5]:
ds_test = datasets.ImageFolder("./transfer_learning/testing", preprocess)

In [6]:
ds_train_ldr = torch.utils.data.DataLoader(ds_train, batch_size=1, shuffle=True)

In [7]:
ds_test_ldr = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=True)

In [8]:
ds_train.imgs

[('./transfer_learning/training/coconut/Cocco-nut_hg.jpg', 0),
 ('./transfer_learning/training/coconut/Coco_2.JPG', 0),
 ('./transfer_learning/training/coconut/Coco_y_composición.jpg', 0),
 ('./transfer_learning/training/coconut/Coconut-60395_-_Hans_Braxmeier.jpg',
  0),
 ('./transfer_learning/training/coconut/coconut.jpg', 0),
 ('./transfer_learning/training/coconut/Coconut_and_oil.jpg', 0),
 ('./transfer_learning/training/coconut/Coconut_drying.jpg', 0),
 ('./transfer_learning/training/durian/ARS_Durian.jpg', 1),
 ('./transfer_learning/training/durian/Ayo_ngambi_Durian_oi........_di_Desa_Jumabalno,_Tiga_Baru,Pegagan_Hilir,_Dairi_-_panoramio.jpg',
  1),
 ('./transfer_learning/training/durian/D101_and_random_stock.jpg', 1),
 ('./transfer_learning/training/durian/durian.png', 1),
 ('./transfer_learning/training/durian/Durian_daf.JPG', 1),
 ('./transfer_learning/training/durian/Durian_fruit_opened_PJ_DSC_0802.jpg',
  1),
 ('./transfer_learning/training/durian/Durio_at_Indonesian_superma

In [9]:
ds_test.imgs

[('./transfer_learning/testing/coconut/coconut1.jpg', 0),
 ('./transfer_learning/testing/coconut/coconut2.jpg', 0),
 ('./transfer_learning/testing/coconut/coconut3.jpg', 0),
 ('./transfer_learning/testing/durian/durian1.png', 1),
 ('./transfer_learning/testing/durian/durian2.jpg', 1),
 ('./transfer_learning/testing/durian/durian3.jpg', 1),
 ('./transfer_learning/testing/lychee/lychee1.jpg', 2),
 ('./transfer_learning/testing/lychee/lychee2.jpg', 2),
 ('./transfer_learning/testing/lychee/lychee3.jpg', 2)]

In [10]:
def train(model, loss_function, optimizer, num_epochs=10):
    """
    Begin training the last layer
    :param model: Model to train
    :param loss_function: Use to assess the model
    :praram optimizer:
    :param num_epochs: Number of epochs to train on.
    :return: best model
    """
    
    
    best_model = model
    best_acc = 0
    
    for e in range(num_epochs):
        l.info("EPOCH: %d/%d", e, num_epochs-1)
        
        for is_training in [True, False]:
            model.train(is_training)
            
            if is_training:
                # Train the model
                for data, target in ds_train_ldr:
                    x, y = Variable(data), Variable(target)
                    
                    optimizer.zero_grad()
                    
                    r = model(x)
                    _, preds = torch.max(r.data, 1)
#                     l.info("Predictions: %s", str(preds))
                    
                    loss = loss_function(r, y)
#                     l.info("Loss: %s", str(loss))
                    
                    loss.backward()
                    optimizer.step()

            else:
                v_loss = 0
                v_acc = 0
                
                # Validate the model
                for data, target in ds_test_ldr:
                    x, y = Variable(data), Variable(target)
                    
                    optimizer.zero_grad()
                    
                    r = model(x)
                    _, preds = torch.max(r.data, 1)
#                     l.info("Predictions: %s", str(preds))
                    
                    loss = loss_function(r, y)
#                     l.info("Loss: %s", str(loss))
                    
                    v_loss += loss.data[0]
                    v_acc += torch.sum(preds == y.data)
                
                epoch_loss = v_loss / len(ds_test)
                epoch_acc = v_acc / len(ds_test)
                
                l.info("Epoch %d, loss %f, acc %f", e, epoch_loss, epoch_acc)
                
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
    l.info("Complete. Best acc %f", best_acc)
    return best_model

Here, we begin loading our pre-train model. I just randomly picked `resnet18`, but you can use [others](http://pytorch.org/docs/master/torchvision/models.html)

In [11]:
# model_conv = torchvision.models.resnet18(pretrained=True)
model_conv = torchvision.models.alexnet(pretrained=True)

Right now, we will only be training the final fully connected layer. We will not be fine-tuning the network.

In [12]:
for param in model_conv.parameters():
    param.requires_grad = False

We will make a new fully connected layer to use for our classification task.

In [18]:
model_conv

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Linear (9216 -> 3)
)

In [13]:
# num_ftrs = model_conv.fc.in_features
num_ftrs = 9216
# model_conv.fc = nn.Linear(num_ftrs, 3)
model_conv.classifier = nn.Linear(num_ftrs, 3)

Nothing fancy here ...

In [14]:
loss_function = nn.CrossEntropyLoss()

In [15]:
optimizer = optim.SGD(model_conv.classifier.parameters(), lr=0.001, momentum=0.9)

In [16]:
model_conv = train(
    model=model_conv, 
    loss_function=loss_function, 
    optimizer=optimizer, 
    num_epochs=100)

[2017-10-31 18:19:21,970][INFO]: EPOCH: 0/99
[2017-10-31 18:19:26,620][INFO]: Epoch 0, loss 1.982643, acc 0.888889
[2017-10-31 18:19:26,626][INFO]: EPOCH: 1/99
[2017-10-31 18:19:31,143][INFO]: Epoch 1, loss 0.000000, acc 1.000000
[2017-10-31 18:19:31,149][INFO]: EPOCH: 2/99
[2017-10-31 18:19:35,653][INFO]: Epoch 2, loss 14.932346, acc 0.777778
[2017-10-31 18:19:35,654][INFO]: EPOCH: 3/99
[2017-10-31 18:19:40,154][INFO]: Epoch 3, loss 15.984222, acc 0.888889
[2017-10-31 18:19:40,155][INFO]: EPOCH: 4/99
[2017-10-31 18:19:44,650][INFO]: Epoch 4, loss 16.168813, acc 0.888889
[2017-10-31 18:19:44,651][INFO]: EPOCH: 5/99
[2017-10-31 18:19:49,145][INFO]: Epoch 5, loss 16.191260, acc 0.888889
[2017-10-31 18:19:49,146][INFO]: EPOCH: 6/99


KeyboardInterrupt: 

## Side note

In Jupyter Notebook, you can use PIL to show the image that you going to be feeding into the network.

In [ ]:
from PIL import Image

In [ ]:
Image.open("chair.jpg")

In [ ]:
prepare_image(Image.open("durian2.jpg"))

In [ ]:
Variable(prepare_image(Image.open("durian2.jpg")))